### 9.1 Wrapper

In [1]:
import time
from functools import wraps

In [2]:
def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [3]:
@timeit
def countdown(n:int):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1
        
countdown(100000)
countdown(1000)
        

countdown 0.01752305030822754
countdown 0.00013709068298339844


In [4]:
countdown.__name__, countdown.__doc__, countdown.__annotations__, countdown.__wrapped__

('countdown',
 '\n    Counts down\n    ',
 {'n': int},
 <function __main__.countdown(n: int)>)

In [5]:
from inspect import signature

print(signature(countdown))

(n: int)
